In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

loading the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone/shuffled_products.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,title,normal_price,label,image_url
0,Cedea Steamboat Set 500G,30%,makanan,https://assets.klikindomaret.com/products/2010...
1,Charm Pembalut Safe Night 7'S Wing 42cm,23%,kesehatan,https://assets.klikindomaret.com/products/2001...
2,Twinings Green Tea Jasmine 10's,Rp 42.700,minuman,https://assets.klikindomaret.com/products/2013...
3,Burung Dara Mie Telur Urai Original 140G,Rp 5.600,makanan,https://assets.klikindomaret.com/products/2008...
4,Indomaret Keripik Bothe Talas Original 50G,Rp 8.200,makanan,https://assets.klikindomaret.com/products/2013...


In [3]:
data = data.drop(columns=[col for col in data.columns if col not in ['title']])
data.head()

,title
0,Cedea Steamboat Set 500G
1,Charm Pembalut Safe Night 7'S Wing 42cm
2,Twinings Green Tea Jasmine 10's
3,Burung Dara Mie Telur Urai Original 140G
4,Indomaret Keripik Bothe Talas Original 50G


In [7]:
def process_data(data, label):
    """ This function removes numbers from a text
        inputs:
         - text """
    # menampung kata kata yang udah bersih
    words=[]

    # remove angka dan satuanya
    data[label] = data[label].apply(lambda x: re.sub(r'(?<=\s)\d+[A-Za-z]+', " ", x))
    # menghapus karakter tidak perlu dalam data
    data[label] = data[label].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    # mengubah nama jadi lowercase
    data[label] = data[label].str.lower()

    words = re.findall(r'\b\w+\b', data[label].str.cat(sep=' '))
    return words

words = process_data(data, "title")
vocab = set(words)
print(f"10 kata awal: \n{words[0:10]}")
print(f"jumlah kata unique: {len(vocab)}")

10 kata awal: 
['cedea', 'steamboat', 'set', 'charm', 'pembalut', 'safe', 'night', 'wing', 'twinings', 'green']
jumlah kata unique: 2092


In [8]:
def get_count(words):
    '''
    Returns a dictionary of word vs frequency of word.
    '''

    word_count_dict = {}

    word_count_dict = Counter(words)
    return word_count_dict

In [9]:
word_count_dict = get_count(words)
print(f"ada {len(word_count_dict)} pasangan key value")
print(f"kata yang paling sering muncul: {word_count_dict.most_common(5)}")

ada 2092 pasangan key value
kata yang paling sering muncul: [('drink', 148), ('susu', 134), ('snack', 132), ('minuman', 124), ('ayam', 76)]


In [10]:
def get_probs(word_count_dict):
    probs = {}

    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    return probs

In [11]:
probs = get_probs(word_count_dict)
print(f"panjang probs: {len(probs)}")
print(f"p(drink) = {probs['drink']:.4f}")

panjang probs: 2092
p(drink) = 0.0146


string manipulation

In [13]:
def delete_letter(word, verbose=False):
  delete_l = []
  split_l = []

  for c in range(len(word)):
    split_l.append((word[:c], word[c:]))
  for a,b in split_l:
    delete_l.append(a + b[1:])

  if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

  return delete_l

In [14]:
delete_word_l = delete_letter(word="susu",
                        verbose=True)

input word susu, 
split_l = [('', 'susu'), ('s', 'usu'), ('su', 'su'), ('sus', 'u')], 
delete_l = ['usu', 'ssu', 'suu', 'sus']


In [16]:
def switch_letter(word, verbose=False):
    switch_l = []
    split_l = []

    word_len=len(word)
    for c in range(word_len):
        split_l.append((word[:c],word[c:]))
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}")

    return switch_l

In [17]:
switch_word_l = switch_letter(word="susu",
                        verbose=True)

Input word = susu 
split_l = [('', 'susu'), ('s', 'usu'), ('su', 'su'), ('sus', 'u')] 
switch_l = ['ussu', 'ssuu', 'suus']


In [19]:
def replace_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'

    replace_l = []
    split_l = []

    for c in range (len(word)):
        split_l.append((word[0:c], word[c:]))
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)
    replace_set.remove(word)

    replace_l = sorted(list(replace_set))

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")

    return replace_l

In [20]:
replace_word_l = replace_letter(word='susu',
                        verbose=True)

Input word = susu 
split_l = [('', 'susu'), ('s', 'usu'), ('su', 'su'), ('sus', 'u')] 
replace_l ['ausu', 'busu', 'cusu', 'dusu', 'eusu', 'fusu', 'gusu', 'husu', 'iusu', 'jusu', 'kusu', 'lusu', 'musu', 'nusu', 'ousu', 'pusu', 'qusu', 'rusu', 'sasu', 'sbsu', 'scsu', 'sdsu', 'sesu', 'sfsu', 'sgsu', 'shsu', 'sisu', 'sjsu', 'sksu', 'slsu', 'smsu', 'snsu', 'sosu', 'spsu', 'sqsu', 'srsu', 'sssu', 'stsu', 'suau', 'subu', 'sucu', 'sudu', 'sueu', 'sufu', 'sugu', 'suhu', 'suiu', 'suju', 'suku', 'sulu', 'sumu', 'sunu', 'suou', 'supu', 'suqu', 'suru', 'susa', 'susb', 'susc', 'susd', 'suse', 'susf', 'susg', 'sush', 'susi', 'susj', 'susk', 'susl', 'susm', 'susn', 'suso', 'susp', 'susq', 'susr', 'suss', 'sust', 'susv', 'susw', 'susx', 'susy', 'susz', 'sutu', 'suuu', 'suvu', 'suwu', 'suxu', 'suyu', 'suzu', 'svsu', 'swsu', 'sxsu', 'sysu', 'szsu', 'tusu', 'uusu', 'vusu', 'wusu', 'xusu', 'yusu', 'zusu']


In [21]:
def insert_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []

    for c in range(len(word)+1):
        split_l.append((word[0:c], word[c:]))
    insert_l = [a + l + b for a,b in split_l for l in letters]

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")

    return insert_l

In [22]:
insert_word_l = insert_letter('susu', True)
print(f"Number of strings output by insert_letter('susu') is {len(insert_letter('susu'))}")

Input word susu 
split_l = [('', 'susu'), ('s', 'usu'), ('su', 'su'), ('sus', 'u'), ('susu', '')] 
insert_l = ['asusu', 'bsusu', 'csusu', 'dsusu', 'esusu', 'fsusu', 'gsusu', 'hsusu', 'isusu', 'jsusu', 'ksusu', 'lsusu', 'msusu', 'nsusu', 'osusu', 'psusu', 'qsusu', 'rsusu', 'ssusu', 'tsusu', 'ususu', 'vsusu', 'wsusu', 'xsusu', 'ysusu', 'zsusu', 'sausu', 'sbusu', 'scusu', 'sdusu', 'seusu', 'sfusu', 'sgusu', 'shusu', 'siusu', 'sjusu', 'skusu', 'slusu', 'smusu', 'snusu', 'sousu', 'spusu', 'squsu', 'srusu', 'ssusu', 'stusu', 'suusu', 'svusu', 'swusu', 'sxusu', 'syusu', 'szusu', 'suasu', 'subsu', 'sucsu', 'sudsu', 'suesu', 'sufsu', 'sugsu', 'suhsu', 'suisu', 'sujsu', 'suksu', 'sulsu', 'sumsu', 'sunsu', 'suosu', 'supsu', 'suqsu', 'sursu', 'sussu', 'sutsu', 'suusu', 'suvsu', 'suwsu', 'suxsu', 'suysu', 'suzsu', 'susau', 'susbu', 'suscu', 'susdu', 'suseu', 'susfu', 'susgu', 'sushu', 'susiu', 'susju', 'susku', 'suslu', 'susmu', 'susnu', 'susou', 'suspu', 'susqu', 'susru', 'sussu', 'sustu', 'susuu'

combining the edits

In [23]:
def edit_one_letter(word, allow_switches = True):
    edit_one_set = set()

    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
        edit_one_set.update(replace_letter(word))
        edit_one_set.update(insert_letter(word))

    return set(edit_one_set)

In [24]:
tmp_word = "susu"
tmp_edit_one_set = edit_one_letter(tmp_word)

tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type od return object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('the') is {len(edit_one_letter('the'))}")

input word susu 
edit_one_l 
['asusu', 'ausu', 'bsusu', 'busu', 'csusu', 'cusu', 'dsusu', 'dusu', 'esusu', 'eusu', 'fsusu', 'fusu', 'gsusu', 'gusu', 'hsusu', 'husu', 'isusu', 'iusu', 'jsusu', 'jusu', 'ksusu', 'kusu', 'lsusu', 'lusu', 'msusu', 'musu', 'nsusu', 'nusu', 'osusu', 'ousu', 'psusu', 'pusu', 'qsusu', 'qusu', 'rsusu', 'rusu', 'sasu', 'sausu', 'sbsu', 'sbusu', 'scsu', 'scusu', 'sdsu', 'sdusu', 'sesu', 'seusu', 'sfsu', 'sfusu', 'sgsu', 'sgusu', 'shsu', 'shusu', 'sisu', 'siusu', 'sjsu', 'sjusu', 'sksu', 'skusu', 'slsu', 'slusu', 'smsu', 'smusu', 'snsu', 'snusu', 'sosu', 'sousu', 'spsu', 'spusu', 'sqsu', 'squsu', 'srsu', 'srusu', 'sssu', 'ssu', 'ssusu', 'ssuu', 'stsu', 'stusu', 'suasu', 'suau', 'subsu', 'subu', 'sucsu', 'sucu', 'sudsu', 'sudu', 'suesu', 'sueu', 'sufsu', 'sufu', 'sugsu', 'sugu', 'suhsu', 'suhu', 'suisu', 'suiu', 'sujsu', 'suju', 'suksu', 'suku', 'sulsu', 'sulu', 'sumsu', 'sumu', 'sunsu', 'sunu', 'suosu', 'suou', 'supsu', 'supu', 'suqsu', 'suqu', 'sursu', 'suru', 'su

In [26]:
def edit_two_letters(word, allow_switches = True):
    edit_two_set = set()
    edit_on = edit_one_letter(word, allow_switches=allow_switches)
    for w in edit_on:
        if w:
            edit_two = edit_one_letter(w, allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    return set(edit_two_set)

In [27]:
tmp_edit_two_set = edit_two_letters("susu")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 24148
First 10 strings ['aasu', 'aasusu', 'aausu', 'absu', 'absusu', 'abusu', 'acsu', 'acsusu', 'acusu', 'adsu']
Last 10 strings ['zwusu', 'zxsu', 'zxsusu', 'zxusu', 'zysu', 'zysusu', 'zyusu', 'zzsu', 'zzsusu', 'zzusu']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


suggest spelling suggestions

In [28]:
# example of logical operation on lists or sets
print( [] and ["a","b"] )
print( [] or ["a","b"] )
#example of Short circuit behavior
val1 =  ["Most","Likely"] or ["Less","so"] or ["least","of","all"]  # selects first, does not evalute remainder
print(val1)
val2 =  [] or [] or ["least","of","all"] # continues evaluation until there is a non-empty list
print(val2)

[]
['a', 'b']
['Most', 'Likely']
['least', 'of', 'all']


In [38]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    suggestions = []
    n_best = []

    suggestions = set()
    suggestions = suggestions.union(edit_one_letter(word))
    suggestions = suggestions.union(edit_two_letters(word))
    suggestions = suggestions.union(delete_letter(word))
    suggestions = suggestions.union(replace_letter(word))
    suggestions = suggestions.union(insert_letter(word))
    suggestions = suggestions.union(switch_letter(word))
    suggestions = suggestions.intersection(vocab)

    probs_suggestions = {s: probs.get(s, 0) for s in suggestions}

    n_best = sorted(probs_suggestions.items(), key=lambda x: x[1], reverse=True)[:n]

    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [64]:
# Test your implementation - feel free to try other words in my word
my_word = 'kcg'
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")

entered word =  kcg 
suggestions =  {'klg', 'kue', 'kit', 'k', 'qc', 'kuo', 'kae', 'cm', 'kin', 'ice', 'dog', 'kat', 'c', 'big', 'egg', 'ku', 'kao', 'wg'}
word 0: dog, probability 0.001781
word 1: kao, probability 0.001484
data type of corrections <class 'list'>
